<a href="https://colab.research.google.com/github/tonystz/cuda/blob/main/poc/gpu-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Get code

In [ ]:
!git clone https://github.com/tonystz/cuda.git

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!python cuda/poc/raw.py

use google drive raw log
[step0]preprocess raw log: total=1153965 skip=12507 3.1145408153533936



###CPU

In [ ]:
!python cuda/poc/cpu.py

[step1]reload data: 1.4388518333435059 (1141458,) |S44 1141458 [b'::1-200' b'::1-200' b'::1-404' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200']
[step2]404 data filter: 0.30924201011657715 202448
[step3]check result: 0.08621692657470703 ['216.244.66.200', '217.174.192.132', '66.249.79.234', '66.249.79.238', '66.249.79.236', '23.96.16.211', '47.101.154.149', '47.101.149.21', '38.54.24.118']


###GPU initialize

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

###GPU

In [ ]:
a=np.loadtxt('pre.log', delimiter=' ', dtype=np.string_,converters={0:lambda v:v+b'\0'})
print(a.shape,a.size,a)

(80,) 80 [b'::1-200' b'::1-200' b'::1-404' b'::1-200' b'183.212.185.122-404'
 b'183.212.185.121-200' b'183.212.185.121-404' b'183.212.185.122-404'
 b'183.212.185.121-200' b'183.212.185.122-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.185.121-304' b'183.212.185.121-304' b'183.212.185.121-304'
 b'183.212.185.121-304' b'183.212.185.121-304' b'183.212.185.121-304'
 b'183.212.185.121-200' b'183.212.185.121-200' b'183.212.185.121-200'
 b'183.212.18

In [ ]:
print(a[0])

b'::1-200'


In [ ]:
na=np.delete(a,0)
na.reshape((479,2383))

array([[b'::1-200', b'::1-404', b'::1-200', ..., b'195.242.110.253-200',
        b'110.249.202.11-200', b'110.249.201.69-200'],
       [b'110.249.201.206-200', b'207.46.13.23-200',
        b'110.249.201.14-200', ..., b'223.104.68.50-200',
        b'223.104.68.50-200', b'223.104.68.50-200'],
       [b'223.104.68.50-200', b'223.104.68.50-200', b'223.104.68.50-200',
        ..., b'125.84.82.12-200', b'125.84.82.12-200',
        b'125.84.82.12-200'],
       ...,
       [b'140.250.200.255-404', b'140.250.200.255-404',
        b'52.167.144.32-200', ..., b'180.164.68.86-200',
        b'180.164.68.86-200', b'180.164.68.86-200'],
       [b'180.164.68.86-200', b'180.164.68.86-200', b'180.164.68.86-200',
        ..., b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200'],
       [b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200', ...,
        b'180.163.28.55-200', b'180.163.28.55-200', b'180.163.28.55-200']],
      dtype='|S44')

In [ ]:
s= np.array(['180.163.28.55-200\0','edfg\0','stz12345678b\0','b1\0','k2\0','hellworld3\0'], dtype=np.string_)

s=s.reshape(2,3)
print('shape:',s.shape,s.size)
print(s,s.dtype)

shape: (2, 3) 6
[[b'180.163.28.55-200' b'edfg' b'stz12345678b']
 [b'b1' b'k2' b'hellworld3']] |S18


In [9]:
from ctypes import *
import sys

if 'linux' in sys.platform:
	cuda = CDLL('libcuda.so')
elif 'win' in sys.platform:
	cuda = CDLL('nvcuda.dll')

cuInit = cuda.cuInit
cuInit.argtypes = [c_uint]
cuInit.restype = int

CU_LIMIT_MALLOC_HEAP_SIZE=2
cuCtxSetLimit = cuda.cuCtxSetLimit
cuCtxSetLimit.argtypes =[c_int,c_size_t]
cuCtxSetLimit.restype = int

print(cuCtxSetLimit(CU_LIMIT_MALLOC_HEAP_SIZE,8388608))


3


In [20]:
%%writefile a.cu
#include <cuda.h>
#include <stdio.h>

int main(int argc, char const *argv[]) {
    // cuda initialization via cudaMalloc
    size_t limit = 0;

    cudaDeviceGetLimit(&limit, cudaLimitMallocHeapSize);
    printf("cudaLimitMallocHeapSize: %u\n", (unsigned)limit);
    cudaDeviceGetLimit(&limit, cudaLimitPrintfFifoSize);
    printf("cudaLimitPrintfFifoSize: %u\n", (unsigned)limit);
    cudaDeviceGetLimit(&limit, cudaLimitMallocHeapSize);
    printf("cudaLimitMallocHeapSize: %u\n", (unsigned)limit);

}

Overwriting a.cu


In [21]:
!nvcc a.cu -o query
!nvprof ./query

==45382== NVPROF is profiling process 45382, command: ./query
cudaLimitMallocHeapSize: 8388608
cudaLimitPrintfFifoSize: 1310720
cudaLimitMallocHeapSize: 8388608
==45382== Profiling application: ./query
==45382== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   99.86%  111.61ms         3  37.202ms     780ns  111.60ms  cudaDeviceGetLimit
                    0.11%  120.35us       101  1.1910us     135ns  50.351us  cuDeviceGetAttribute
                    0.02%  25.205us         1  25.205us  25.205us  25.205us  cuDeviceGetName
                    0.01%  7.0620us         1  7.0620us  7.0620us  7.0620us  cuDeviceGetPCIBusId
                    0.00%  2.0930us         3     697ns     216ns  1.6050us  cuDeviceGetCount
                    0.00%     975ns         2     487ns     267ns     708ns  cuDeviceGet
                    0.00%     463ns         1     463ns     463ns     463ns  cuDeviceTotalMem
 

In [82]:
#!grep '404' pre.log
#!nvcc --cubin -arch sm_75 /tmp/tmp91fglzn7/kernel.cu
!cd cuda && git pull
!cd cuda/poc && make


Already up to date.
nvcc -ptx -o kernel.ptx kernel.cu
python gpu.py
shape: (1141457,) 1141457 |S44
[b'::1-200' b'::1-404' b'::1-200' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200'] <memory at 0x7fc40795e340>
Traceback (most recent call last):
  File "/content/cuda/poc/gpu.py", line 31, in <module>
    pycuda.cuCtxGetLimit
AttributeError: module 'pycuda' has no attribute 'cuCtxGetLimit'
make: *** [Makefile:3: all] Error 1
